In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# MERGE

Merging is for doing complex column-wise combinations of dataframes in an SQL-like way

In [3]:
tips_bill = tips.groupby(["sex", "smoker"])[["total_bill", "tip"]].sum()
tips_tip = tips.groupby(["sex", "smoker"])[["total_bill", "tip"]].sum()

del tips_bill["tip"]
del tips_tip["total_bill"]

In [4]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [5]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

In [6]:
pd.merge?

In [7]:
# merge on the indexes
pd.merge(
    tips_bill, 
    tips_tip, 
    right_index=True, 
    left_index=True)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [8]:
# we can reset indexes and then merge on the column
pd.merge(
    tips_bill, 
    tips_tip, 
    on=["smoker", "sex"])

,,total_bill,tip
smoker,sex,,
Yes,Male,1337.07,183.07
No,Male,1919.75,302.00
Yes,Female,593.27,96.74
No,Female,977.68,149.77


In [9]:
tips_bill.reset_index()

,sex,smoker,total_bill
0,Male,Yes,1337.07
1,Male,No,1919.75
2,Female,Yes,593.27
3,Female,No,977.68


In [10]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index(), 
    on=["smoker", "sex"])

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [11]:
# the above can be inferred, but we need to be careful
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index())

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [12]:
# it can do interesting combinations
tips_bill_strange = tips_bill.reset_index(level=0)
tips_bill_strange

,sex,total_bill
smoker,,
Yes,Male,1337.07
No,Male,1919.75
Yes,Female,593.27
No,Female,977.68


In [13]:
pd.merge(
    tips_tip, 
    tips_bill_strange, 
    on=["smoker", "sex"])

,sex,tip,total_bill
smoker,,,
Yes,Male,183.07,1337.07
No,Male,302.00,1919.75
Yes,Female,96.74,593.27
No,Female,149.77,977.68


In [14]:
pd.merge(
    tips_tip, 
    tips_bill_strange, 
    on=["smoker", "sex"]).reset_index()

,smoker,sex,tip,total_bill
0,Yes,Male,183.07,1337.07
1,No,Male,302.00,1919.75
2,Yes,Female,96.74,593.27
3,No,Female,149.77,977.68


In [15]:
pd.merge(
    tips_tip.reset_index(), 
    tips_bill_strange, 
    on=["smoker", "sex"])

,sex,smoker,tip,total_bill
0,Male,Yes,183.07,1337.07
1,Male,No,302.00,1919.75
2,Female,Yes,96.74,593.27
3,Female,No,149.77,977.68


In [16]:
# we can do any SQL-like functionality
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2), 
    how="left")

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [17]:
pd.merge(
    tips_bill.reset_index(), 
    tips_tip.reset_index().head(2), 
    how="inner")

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00


In [18]:
# With indicator added
pd.merge(
    tips_bill.reset_index().tail(4), 
    tips_tip.reset_index().head(3), 
    how="outer",
    indicator=True)

,sex,smoker,total_bill,tip,_merge
0,Male,Yes,1337.07,183.07,both
1,Male,No,1919.75,302.00,both
2,Female,Yes,593.27,96.74,both
3,Female,No,977.68,NaN,left_only


In [19]:
# It can handle columns with the same name
pd.merge(
    tips_bill, 
    tips_bill,
    right_index=True,
    left_index=True,
    suffixes=("_left", "_right"))

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

In [20]:
pd.merge(
    tips_bill.head(2), 
    tips_bill.head(4),
    right_index=True,
    left_index=True,
    suffixes=("_left", "_right"),
    how="outer")

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes                 NaN            593.27
       No                  NaN            977.68

# CONTATENATE

Contatenating is for combining more than two dataframes in either column-wise or row-wise. The problem with concatenate is that the combinations it allows you to do are rather simplistic. That's why we need merge.

Concatenate can take as many data frames as you want, but it requires that they are specifically constructed. All of the dataframes you pass in will need to have the same index. So no more using columns as an index.

In [21]:
# this adds the datafames together row-wise
pd.concat([tips_bill, tips_bill, tips_tip], sort=False)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [22]:
# this does it column-wise
pd.concat([tips_bill, tips_bill.head(3), tips_tip, tips_tip.tail(2)], axis=1)

total_bill  total_bill     tip     tip
sex    smoker                                        
Female No          977.68         NaN  149.77  149.77
       Yes         593.27      593.27   96.74   96.74
Male   No         1919.75     1919.75  302.00     NaN
       Yes        1337.07     1337.07  183.07     NaN

In [23]:
# and finallythis will add on the dataset where it's from
pd.concat([tips_bill, tips_tip], sort=False, keys=["num0", "num1"])

total_bill     tip
     sex    smoker                    
num0 Male   Yes        1337.07     NaN
            No         1919.75     NaN
     Female Yes         593.27     NaN
            No          977.68     NaN
num1 Male   Yes            NaN  183.07
            No             NaN  302.00
     Female Yes            NaN   96.74
            No             NaN  149.77